# django 불러오기

In [ ]:
import os, sys
PWD = os.getenv('PWD')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [ ]:
init_django('mydjango_project')

# model import

In [ ]:
from board.models import Board, Comment

# Insert or Update

In [ ]:
board = Board(
    title="게시글1",
    content="내용1",
)

In [ ]:
board.save()

In [ ]:
board

-> 내용 확인

In [ ]:
board = Board.objects.create(
    title="게시글2",
    content="내용2"
)

In [ ]:
board

In [ ]:
board = Board.objects.create(
    title="게시글3",
    content="내용3"
)

In [ ]:
board

# 조회하기

## raw query로 조회하기

In [ ]:
result = Board.objects.raw('SELECT * FROM board_board')
result

## orm으로 조회하기

### 전체 조회하기

In [ ]:
board_list = Board.objects.all()
board_list

### filter 걸어서 조회하기(where)

In [ ]:
board_list = Board.objects.filter(title="게시글")
board_list

In [ ]:
board_list = Board.objects.order_by('-created_at') # order by desc
board_list

In [ ]:
board_list = Board.objects.order_by('-created_at')[2:4] # order by desc limit(2) offset(2)
board_list

###  filter not 표시

In [ ]:
board_list = Board.objects.exclude(title="게시글")
board_list

In [ ]:
from django.db.models import Q

In [ ]:
board_list = Board.objects.filter(~Q(title="게시글"))
board_list

## 조회하기 고급

### 다양한 조건

#### or

In [ ]:
board_list = Board.objects.filter(Q(title="게시글1") | Q(title="게시글2"))
board_list

##### and 와 연산자

- gte: greater than equal
- lte: less than equal
- lt: less than
- gt: greater than

In [ ]:
board_list = Board.objects.filter(Q(title__endswith="글1") & Q(id__gte=3))
board_list

In [ ]:
board_list = Board.objects.filter(Q(title__startswith="게시") & Q(created_at__year=2022))
board_list

# 관계 추가하기

1번 댓글에 댓글을 3가지 이상 입력하세요

In [ ]:
comment = Comment(
    board_id = 1,
    content = "댓글 내용2"
)
comment.save()

In [ ]:
board = Board.objects.get(id=1)
comment = Comment.objects.create(
    board=board,
    content = "댓글 내용3"
)

## 관계 조회하기

In [ ]:
Comment.objects.all()

In [ ]:
board_list = Board.objects.filter(id__lte=5).prefetch_related('comment_set')
board_list

In [ ]:
comment_list = board_list[0].comment_set.all()
comment_list

In [ ]:
comment = comment_list[0]
comment.board